In [2]:
from zizou.data import DataSource, GeoMagWaveforms
from zizou.ssam import SSAM 
from datetime import datetime, date, timedelta, timezone
import pandas as pd
import requests
from tonik import Storage
from io import StringIO
from obspy import UTCDateTime, Trace
import numpy as np
from zizou.visualise import plot_ssam_plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from zizou.dsar import DSAR
from zizou.rsam import RSAM, EnergyExplainedByRSAM
from zizou.ssam import SSAM
from zizou.spectral_features import SpectralFeatures
from zizou.data import DataSource, MockSDSWaveforms
from obspy import UTCDateTime

import os 

In [ ]:
# gmc = GeoMagWaveforms(base_url='https://tilde.geonet.org.nz/v3/data',
#                       method='60s', aspect='F-total-field', name='magnetic-field')
gmc = GeoMagWaveforms(base_url='https://tilde.geonet.org.nz/v3/data',
                      method='60s', aspect='H-horizontal-intensity', name='magnetic-field')
ssam = SSAM(per_lap=0.5, timestamp='start', interval=1024*60.,
            frequencies=np.linspace(1/1000., 1/120., 50))

rsam = RSAM(filtertype="bp", filterfreq=(0.001, 0.1), interval=1024*60.)
spec = SpectralFeatures(filtertype="bp", filterfreq=(0.001, 0.1), interval=1024*60.)
dsar = DSAR(lowerfreqband=(0.0001, 0.001), higherfreqband=(0.001, 0.008), interval=1024*60.)
#dsar = DSAR(lowerfreqband=(0.01, 1), higherfreqband=(1, 2.5), interval=1024*60.)
rsam_energy_prop = EnergyExplainedByRSAM(filterfreq_wb=(0.001,0.1))

ds = DataSource(clients=[gmc])
sg = Storage('EYR', rootdir='/tmp/geomag/features')
st = sg.get_substore('EYWM', '50', 'H')


startdate = UTCDateTime(2024, 5, 6)
#enddate = UTCDateTime.now()
#enddate = UTCDateTime(2024, 5, 12)
enddate = UTCDateTime(2024, 10, 20)



for tr in ds.get_waveforms(net='NZ', site='EYWM', loc='50', comp='H',
                           start=startdate, end=enddate,
                           cache=True):
    print(tr)
    # for feat in (ssam, rsam, spec):
    #     st.save(feat.compute(trace=tr))
    
    for feat in (dsar, spec, ssam, rsam, rsam_energy_prop):
        st.save(feat.compute(trace=tr))


    # xds = ssam.compute(tr) 

    # xds = ssam.compute(tr) 
    # st.save(xds)

    # for tr in ds.get_waveforms(net="NZ", site=site, loc=sensor, comp=channel, start=UTCDateTime("2024-05-01"), end=UTCDateTime("2024-06-01")):
    #     print(tr)
    #     for feat in (rsam, rsam_energy_prop, ssam, dsar, spec):
    #         store.save(feat.compute(trace=tr))



In [19]:
st.starttime = startdate.datetime
st.endtime = enddate.datetime
ssam = st('ssam')
rsam = st('rsam')
filtbank = st('filterbank')


In [ ]:
filtbank.data.shape

In [ ]:
import matplotlib.pyplot as plt 
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 5), sharex=True)
ax1.plot(rsam.coords["datetime"], rsam.data)
ax1.set_ylabel("RSAM")
ax2.imshow(ssam.data, extent=[rsam.coords["datetime"][0].values, rsam.coords["datetime"][-1].values, 0, 5])
ax2.set_aspect("auto")
ax2.set_ylabel("SSAM")

In [ ]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
tr = next(ds.get_waveforms(net='NZ', site='EYWM', loc='50', comp='F',
                           start=startdate, end=enddate))
dt = [startdate.datetime + timedelta(seconds=s) for s in tr.times()]
fig.add_trace(go.Scatter(x=dt, y=tr.data, mode='lines'), row=1, col=1)
fig.add_trace(plot_ssam_plotly(filtbank, dbscale=True, new_fig=False), row=2, col=1)

### Application of autoencoder and clustuirng (using built in autoencoder in Zizou)


In [ ]:
import datetime

from obspy import UTCDateTime
import numpy as np

from zizou.autoencoder import AutoEncoder

#model configuration
config = """
autoencoder: 
    layers: [2000,500,200,6]
    epochs: 5
    patience: 5
"""

#feature request: 
sg = Storage('EYR', rootdir='/tmp/geomag/features')
st = sg.get_substore('EYWM', '50', 'H')
st.station = 'EYWM.50.H'
st.starttime = startdate.datetime
st.endtime = enddate.datetime
model = AutoEncoder(st, configfile=config, batch_size=32, features=['filterbank'])
model.clear()
# -- Train the Auto-Encoder and save the model (this might take a while)
classifications = model.fit_transform(startdate.datetime, enddate.datetime)





In [25]:
st.starttime = startdate.datetime
st.endtime = enddate.datetime
ac_loss = st('autoencoder_loss')

In [ ]:
fig = make_subplots(rows=3, cols=1, shared_xaxes=True)
tr = next(ds.get_waveforms(net='NZ', site='EYWM', loc='50', comp='F',
                           start=startdate, end=enddate))
dt = [startdate.datetime + timedelta(seconds=s) for s in tr.times()]
fig.add_trace(go.Scatter(x=pd.to_datetime(ac_loss.datetime), y=np.log10(ac_loss.data), mode='lines'), row=1, col=1)
fig.add_trace(go.Scatter(x=dt, y=tr.data, mode='lines'), row=2, col=1)
fig.add_trace(plot_ssam_plotly(filtbank, dbscale=True, new_fig=False), row=3, col=1)

### Testing developing LSTM Autoencoder and clusturing => new version 
Autoencoders for anomaly detection in time series data. For clustering and anomaly detection in time series data, you can combine an LSTM Autoencoder for feature extraction and K-means or other clustering algorithms for identifying anomalous patterns.

Steps of LSTM Autoencoder for time series anomaly detection and clustering:

    Data preparation (sliding window method for time series).
    LSTM Autoencoder Model (for feature extraction).
    Train the autoencoder and use reconstruction error for anomaly detection.
    Clustering the latent features using K-means or other clustering methods.

In [13]:
# 1. Data preperation
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

# time series data
tr = next(ds.get_waveforms(net='NZ', site='EYWM', loc='50', comp='H', start=startdate, end=enddate))
dt = [startdate.datetime + timedelta(seconds=s) for s in tr.times()]
time_series1 = tr.data
time_series2 = ssam.data
time_series3 = filtbank.data

# # Stack the time series into a 2D array where each column is a feature
# time_series_multi = np.vstack((time_series1, time_series2)).T  # Shape: (1000, 2)

# # Normalize each feature independently
# scaler = MinMaxScaler(feature_range=(-1, 1))
# time_series_scaled = scaler.fit_transform(time_series_multi)


# Normalize the data
scaler = MinMaxScaler(feature_range=(-1, 1))
time_series_scaled = scaler.fit_transform(time_series3.reshape(-1, 1)).reshape(-1)

#Create sliding windows for time series forecasting
def create_sequences(data, seq_length):
    sequences = []
    for i in range(len(data) - seq_length):
        seq = data[i:i + seq_length]
        sequences.append(seq)
    return np.array(sequences)

seq_length = 120
X = create_sequences(time_series_scaled, seq_length)

# Convert to PyTorch tensors
X_train = torch.tensor(X, dtype=torch.float32).unsqueeze(-1)


In [16]:
# 2. LSTM Autoencoder model
class LSTM_Autoencoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTM_Autoencoder, self).__init__()
        
        # Encoder
        self.encoder_lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        
        # Decoder
        self.decoder_lstm = nn.LSTM(hidden_size, input_size, num_layers, batch_first=True)

    def forward(self, x):
        # Encoder
        _, (hidden, _) = self.encoder_lstm(x)  # Only hidden state is used
        hidden = hidden[-1].unsqueeze(0).repeat(x.size(1), 1, 1)  # Repeat hidden state
        
        # Decoder
        decoded, _ = self.decoder_lstm(hidden)
        return decoded

# Hyperparameters
input_size = 1
hidden_size = 64
num_layers = 2

model = LSTM_Autoencoder(input_size, hidden_size, num_layers)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [ ]:
#train the autoencoder: 
num_epochs = 10
model.train()

# In your training loop, make sure the input is [batch_size, seq_len, input_size]
# If necessary, transpose the input and target before feeding into the model

for epoch in range(num_epochs):
    # Ensure input shape is [batch_size, seq_len, input_size]
    inputs = X_train # Transpose batch and sequence dimensions if needed
    
    outputs = model(inputs)  # Model forward pass
    
    # Ensure the target is also in the right shape
    targets = X_train.transpose(0, 1)  # Targets should match input shape
    
    loss = criterion(outputs, targets)  # Calculate loss
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 5 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Once the model is trained, we can use the reconstruction error (the difference between the input and the reconstructed output) to detect anomalies. High reconstruction error indicates an anomaly.

In [ ]:
#anomaly detection: 
# Switch to evaluation mode
model.eval()

# Reconstruct the time series or ssam
with torch.no_grad():
    reconstructed = model(X_train).cpu().numpy()

reconstructed_transposed = np.transpose(reconstructed, (1, 0, 2))
# Calculate the reconstruction error
reconstruction_errors = np.mean(np.abs(reconstructed_transposed - X_train.cpu().numpy()), axis=1)

# Set an anomaly detection threshold based on the reconstruction error
threshold = np.percentile(reconstruction_errors, 95)  # 95th percentile of reconstruction errors

# Detect anomalies
anomalies = reconstruction_errors > threshold

# Plot the reconstruction errors and anomalies
plt.figure(figsize=(15, 6))
plt.plot(reconstruction_errors, label='Reconstruction Errors')
plt.axhline(y=threshold, color='r', linestyle='--', label='Threshold')
plt.scatter(np.where(anomalies)[0], reconstruction_errors[anomalies], color='red', label='Anomalies')
plt.title('Reconstruction Errors and Detected Anomalies')
plt.xlabel('Time Step')
plt.ylabel('Reconstruction Error')
plt.legend()
plt.show()



In [ ]:
sample_idx

In [ ]:
import matplotlib.pyplot as plt

# Assume `X_test` is the original input data and `reconstructed_test` is the model's output
# Convert tensors to numpy arrays
X_train_np = X_train.cpu().numpy()
reconstructed_train_np = reconstructed_transposed

# Plot a sample (e.g., the first time series from the test set)
sample_idx = 8000  # Choose a sample index to visualize

plt.figure(figsize=(10, 6))
plt.plot(X_train_np[sample_idx], label='Original', linestyle='-', marker='o')
plt.plot(reconstructed_train_np[sample_idx], label='Reconstructed', linestyle='--', marker='x')
plt.title(f'Original vs Reconstructed for Sample {sample_idx}')
plt.legend()
plt.show()


The LSTM autoencoder's hidden states can serve as feature vectors for clustering. You can apply K-means clustering on these latent features to group similar time series patterns.

In [ ]:
#Step 5: Cluster Latent Features
from sklearn.cluster import KMeans

# Extract hidden states from the encoder as features for clustering
def get_hidden_features(model, X):
    model.eval()
    with torch.no_grad():
        _, (hidden, _) = model.encoder_lstm(X)
        features = hidden[-1].cpu().numpy()
    return features

# Get latent features from the encoder
latent_features = get_hidden_features(model, X_train)

# Apply K-means clustering
n_clusters = 2
kmeans = KMeans(n_clusters=n_clusters, n_init=10, random_state=42)
cluster_labels = kmeans.fit_predict(latent_features)

# Plot clusters
plt.figure(figsize=(10, 5))
plt.scatter(np.arange(len(cluster_labels)), cluster_labels, c=cluster_labels, cmap='viridis', marker='o')
plt.title('K-means Clustering of Latent Features')
plt.xlabel('Time Step')
plt.ylabel('Cluster Label')
plt.show()


### Sapi's testing 2

In [15]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
tr = next(ds.get_waveforms(net='NZ', site='EYWM', loc='50', comp='H',
                           start=startdate, end=enddate))
dt = [startdate.datetime + timedelta(seconds=s) for s in tr.times()]


In [16]:
time_series = tr.data
time_steps = 3600 # Length of each segment
step = 1         # Step size (1 means fully overlapping segments)
n_features = 1   # Number of features (e.g., one magnetogram channel)



In [17]:
def create_segments(data, time_steps, step, n_features):
    X, y = [], []
    for i in range(0, len(data) - time_steps, step):
        segment = data[i:i + time_steps]
        X.append(segment)
        y.append(data[i + time_steps])  # Predicting the next value as the target
    X = np.array(X)
    y = np.array(y)
    X = X.reshape((X.shape[0], X.shape[1], n_features))  # Reshape to [samples, time_steps, n_features]
    return X, y

X, y = create_segments(time_series, time_steps, step, n_features)


In [ ]:
from sklearn.model_selection import train_test_split
import torch

# Split into training and testing sets - 80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check shapes
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

# Convert to PyTorch tensors
# X_train = torch.tensor(X_train, dtype=torch.float32)
# y_train = torch.tensor(y_train, dtype=torch.float32)
# X_test = torch.tensor(X_test, dtype=torch.float32)
# y_test = torch.tensor(y_test, dtype=torch.float32)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1)

In [22]:
#Define the LSTIM model in pytorch => need to test this
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

# Modified LSTM model to return hidden states as feature vectors
class LSTMFeatureExtractor(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(LSTMFeatureExtractor, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, (hn, cn) = self.lstm(x, (h0, c0))
        return hn[-1]  # Use the last hidden state as the feature vector

        

# Define model parameters
input_size = n_features  # Number of features
hidden_size = 10      # Number of hidden units in each LSTM layer
num_layers = 5     # Number of LSTM layers
#output_size = 1         # Size of the feature vector

# Initialize the model
model = LSTMFeatureExtractor(input_size, hidden_size, num_layers)

model = LSTMFeatureExtractor(input_size, hidden_size, num_layers)
model.eval()

# Extract feature vectors from the LSTM model
with torch.no_grad():
    feature_vectors = model(X_train).cpu().numpy()  # Feature vectors of shape (batch_size, hidden_size)



In [ ]:
#clusturing feature vectors

# Number of clusters you want to create
n_clusters = 2

# Apply K-means clustering
kmeans = KMeans(n_clusters=n_clusters, n_init=20, random_state=42)
clusters = kmeans.fit_predict(feature_vectors)

# Plot the clusters for visualization (only if the feature dimension is 2 or 3)
plt.scatter(feature_vectors[:, 0], feature_vectors[:, 1], c=clusters, cmap='viridis')
plt.title('Clusters of LSTM Feature Vectors')
plt.xlabel('Feature Dimension 1')
plt.ylabel('Feature Dimension 2')
plt.show()

In [ ]:
# Assuming 'clusters' contains the cluster labels for each segment
# 'X_train' contains the original segments of the time series

# Example: Assigning cluster labels to each segment
labeled_segments = list(zip(X_train, clusters))

# Example function to map clusters back to the original waveform
def map_clusters_to_waveform(time_series, time_steps, step, clusters):
    cluster_mapping = np.full(len(time_series), np.nan)  # Initialize with NaNs
    segment_start = 0
    
    for i, cluster_label in enumerate(clusters):
        segment_end = segment_start + time_steps
        cluster_mapping[segment_start:segment_end] = cluster_label
        segment_start += step  # Move by the step size
    
    return cluster_mapping

# Assuming you have the original time series 'time_series' and cluster labels 'clusters'
cluster_mapping = map_clusters_to_waveform(time_series, time_steps, 1, clusters)

# Plot the original waveform with cluster labels
plt.figure(figsize=(15, 6))
plt.plot(time_series, label='Original Waveform')
plt.scatter(np.arange(len(time_series)), cluster_mapping * max(time_series), color='red', label='Cluster Labels', marker='o')
plt.title('Waveform with Clustered Events')
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.legend()
plt.show()



In [ ]:
#Model training
# Training
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 20
train_losses = []
test_losses = []

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    train_loss = criterion(outputs, y_train)
    train_loss.backward()
    optimizer.step()
    
    # Store training loss
    train_losses.append(train_loss.item())
    
    # Validation/Test loss
    model.eval()
    with torch.no_grad():
        test_outputs = model(X_test)
        test_loss = criterion(test_outputs, y_test)
        test_losses.append(test_loss.item())

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss.item():.4f}, Test Loss: {test_loss.item():.4f}')

# # Plotting
# plt.figure(figsize=(10, 5))
# plt.plot(train_losses, label='Train Loss')
# plt.plot(test_losses, label='Test Loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.title('Model Loss Progression')
# plt.legend()
# plt.show()

In [ ]:
#Model testing
# Testing
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    test_loss = criterion(test_outputs, y_test)

print(f'Test Loss: {test_loss.item():.4f}')